# Ratajczyk

Lab 4

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def process(frame: np.ndarray) -> np.ndarray:
    frame = ((frame>40)*255).astype(np.uint8)
    frame = cv2.medianBlur(frame, 5)
    frame = cv2.morphologyEx(frame, cv2.MORPH_OPEN, np.ones((11, 11), np.uint8))
    frame = cv2.morphologyEx(frame, cv2.MORPH_CLOSE, np.ones((27, 3), np.uint8))
    return frame

In [3]:
def index(frame: np.ndarray, processed: np.ndarray) -> float:
    image = frame.copy()
    _, _, stats, centroids = cv2.connectedComponentsWithStats(processed)
    if stats.shape[0] > 1:
        for i in filter(lambda x: stats[x, 2] < stats[x, 3], range(1, stats.shape[0])):
            pi = i
            cv2.rectangle(image, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2], stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(image,f"{stats[pi,4]}", (stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))
            cv2.putText(image, f"{pi}", (np.int(centroids[pi,0]),np.int(centroids[pi,1])),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))
    return image

In [4]:
def generate_frames(frame: np.ndarray, processed: np.ndarray, idx: int) -> None:
    _, _, stats, _ = cv2.connectedComponentsWithStats(processed)
    if stats.shape[0] > 1:
        for pi in filter(lambda x: stats[x, 2] < stats[x, 3], range(1, stats.shape[0])):
            try:
                cv2.imwrite(f'patterns/sample_{idx}.png',  frame[stats[pi,1]:stats[pi,1]+stats[pi,3], stats[pi,0]:stats[pi,0]+stats[pi,2]])
            except:
                pass
    idx += 1
    return idx

In [9]:
cap = cv2.VideoCapture('data/vid1_IR.avi')
idx= 0 
while(cap.isOpened()):
    ret, frame = cap.read()
    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    cv2.imshow('IR', index(G, process(G)))
    idx = generate_frames(G, process(G), idx)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()
cap.release()


C:\Users\igor\AppData\Local\Temp\ipykernel_14288\574386502.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cv2.putText(image, f"{pi}", (np.int(centroids[pi,0]),np.int(centroids[pi,1])),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))


In [6]:
y +6

NameError: name 'y' is not defined

In [10]:
from os.path import join
import cv2
import numpy as np


def detect(net, img):
    size = img.shape
    height = size[0]
    width = size[1]
    blob = cv2.dnn.blobFromImage(img, 1 / 255, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)
    boxes = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
    return boxes

def filter_boxes(boxes):
    all_paired_boxes = list()
    for ii, box1 in enumerate(boxes):
        x1, y1, w1, h1 = box1
        center_x1 = x1 + int(w1 / 2)
        center_y1 = y1 + int(h1 / 2)
        to_connect = [ii]
        for jj, box2 in enumerate(boxes):
            if jj != ii:
                x2, y2, w2, h2 = box2
                center_x2 = x2 + int(w2 / 2)
                center_y2 = y2 + int(h2 / 2)
                if abs(center_x2 - center_x1) < 10 and abs(center_y2 - center_y1) < 10:
                    to_connect.append(jj)
        all_paired_boxes.append(to_connect)
    all_paired_boxes = sorted(all_paired_boxes, key=lambda x: len(x), reverse=True)
    all_paired = list()
    final_boxes = list()
    for conn in all_paired_boxes:
        if all([a not in all_paired for a in conn]):
            for a in conn:
                all_paired.append(a)
            final_boxes.append(conn)
    out_boxes = [[int(sum([boxes[i][a] for i in elem]) / len(elem)) for a in range(4)] for elem in final_boxes]
    return out_boxes

def IoU(rect1, rect2):
    x1, y1, w1, h1 = rect1
    x2, y2, w2, h2 = rect2
    left = max([x1, x2])
    right = min([x1+w1, x2+w2])
    top = max([y1, y2])
    bottom = min([y1+h1, y2+h2])
    area1 = max([(right - left), 0]) * max([(bottom - top), 0])
    area2 = (w1 * h1) + (w2 * h2) - area1
    IoU = area1/area2
    return IoU

############# METHOD ###############
# Wybierz metodę fuzji
FUSION = "LATE"
# FUSION = "EARLY"

############# TODO0 ###############
# Ustaw ścieżki
test_rgb = r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab4\YOLO\YOLO\test_rgb"  # Ścieżka do foldeu test_rgb
test_thermal = r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab4\YOLO\YOLO\test_thermal"  # Ścieżka do foldeu test_thermal
###################################

net_fus = None
net_therm = None
net_rgb = None
if FUSION == "EARLY":
    net_fus = cv2.dnn.readNet(r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab4\YOLO\YOLO\yolov3_training_last_f.weights", r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab4\YOLO\YOLO\yolov3_testing_f.cfg")
if FUSION == "LATE":
    net_therm = cv2.dnn.readNet(r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab4\YOLO\YOLO\yolov3_training_last_t.weights", r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab4\YOLO\YOLO\yolov3_testing_t.cfg")
    net_rgb = cv2.dnn.readNet(r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab4\YOLO\YOLO\yolov3_training_last_c.weights", r"C:\Users\igor\Desktop\Studia\I\Semestr6\ZAW\Lab4\YOLO\YOLO\yolov3_testing_c.cfg")

for i in range(200, 300): # można zmieniać range do 518
    path_rgb = join(test_rgb, f"img{i}.png")
    path_thermal = join(test_thermal, f"img{i}.png")
    img_rgb = cv2.imread(path_rgb)
    img_thermal = cv2.imread(path_thermal)
    img_thermal = cv2.cvtColor(img_thermal, cv2.COLOR_BGR2GRAY)
    out_img = None
    boxes = None
    if FUSION == "EARLY":
        ############ TODO1 ##################
        # Połącz RGB z Thermal podążając za instrukcją
        # 1. Utwórz nową ramkę (numpy array) o wymiarach zdjęcia RGB i nazwij ją "new_fus"
        new_fus = np.zeros(img_rgb.shape)
        # 2. Skopiuj dwa pierwsze kanały z RGB (img_rgb[:, :, :2]) do dwóch pierwszych kanałów nowej ramki (new_fus[:, :, :2])
        new_fus[:, :, :2] = img_rgb[:, :, :2]
        # 3. Wartość trzeciego kanału nowej ramki to maksimum z wartości trzeciego kanału RGB i obrazu z termowizji (jednokanałowego)0
        new_fus[:, :, 2] = np.maximum(img_rgb[:, :, 2], img_thermal[:, :])
        #    Użyj w tym celu np.maximum(a, b). Gdzie a i b to 3 kanał RGB oraz termowizja
        # 4. Zrzutuj "new_fus" na "uint8" (new_fus.astype("uint8"))
        new_fus = new_fus.astype(np.uint8)

        ####################################
        out_img = new_fus
        boxes = detect(net_fus, new_fus)
    if FUSION == "LATE":
        out_img = img_rgb
        Rect1 = detect(net_therm, img_thermal)
        Rect2 = detect(net_rgb, img_rgb)
        boxes = None
        boxes_iou = []
        for i in range(len(Rect1)):
            for j in range(len(Rect2)):
                iou = IoU(Rect1[i], Rect2[j])
                if iou > 0: boxes_iou.append([(i, j), iou])
        boxes_iou = sorted(boxes_iou, key=lambda a: a[1], reverse=True)

        Rect1_paired = []
        Rect2_paired = []
        paired_boxes = []

        for elem in boxes_iou:
            if elem[0][0] not in Rect1_paired and elem[0][1] not in Rect2_paired:
                paired_boxes.append(elem[0])
                Rect1_paired.append(elem[0][0])
                Rect2_paired.append(elem[0][1])
        
        boxes = []
        for elem in paired_boxes:
            r1 = Rect1[elem[0]]
            r2 = Rect2[elem[1]]
            avg_r = [int((r1[0]+r2[0])/2), int((r1[1]+r2[1])/2), int((r1[2]+r2[2])/2), int((r1[3]+r2[3])/2)]
            boxes.append(avg_r)

        ######################################
    out_boxes = filter_boxes(boxes)
    for box in out_boxes:
        x, y, w, h = box
        cv2.rectangle(out_img, (x, y), (x+w, y+h), (255, 255, 0), 2)
    cv2.imshow('Image', out_img)
    cv2.waitKey(10)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()
cap.release()